# 지하철 빠른 길 찾기

기능 :
출발역에서 도착역까지의 소요시간과 경로를 나타낸다.

필요한 것

메서드 : bfs, visitStation

지하철들과 다음 역까지의 소요시간이 표현되어있는 딕셔너리(stations)

기능구현의 핵심 :

메인함수에서

1 소요시간이 가장 짧은 역부터 긴 역 순으로 방문하고 방문한 역은 visited을 1로 초기화하여 표시한다.

2 새로운 역을 방문하면 그 역과 이어진 역들까지의 소요시간을 수정한다. 단, 이전에 이미 최소 소요시간이 구해졌다면 소요시간을 서로 비요하여 작은 것으로 바꾸거나 유지한다.
visitStation메서드는 현재 역과 연결된 역들과의 시간을 계산하고 시간이 더 짧은 것으로 업데이트를 한다.

3 그래프의 모든 역들을 방문할 때 까지 3, 4과정을 반복한다.






In [1]:
#BFS로 순회하며 지하철 역을 저장하는 visited 리스트를 채우는 메서드. 큐를 이용하여 순회한다. 
def bfs(graph, start):
    visited = []
    queue = [start]
    
    while queue:
        n = queue.pop(0)
        if n not in visited:
            visited.append(n)
            queue.extend(graph[n].keys())
    return visited
        
    
#2
#방문시 flag on  -> S집합에 포함시키기. + 방문한 역의 다음 역들의 소요시간 정보를 update한다.
def visitStation(visit):
    #flag on.
    routing[visit]['visited'] = 1 #visit에 해당하는 역의 routing 딕셔너리의 키 값이 visited인 것의 vlaue를 1로 바꾼다.
    for toGo, betweenDist in stations[visit].items(): #toGo : visit의 다음 역, betweenDist : 다음 역까지의 소요시간 
        toDist = routing[visit]['shortestTime'] + betweenDist #toDist = 목적지까지의 시간(지금까지의 최소소요시간에 다음역까지의 소요시간의 합)
        if (routing[toGo]['shortestTime'] >= toDist) or  not routing[toGo]['route']: #만약 다음 역까지의 기존 최소소요시간보다 목적지까지의 시간이 더 작거나 || 다음역이 이미 지나간(route) 내역에 포함되지 않을 경우
            routing[toGo]['shortestTime'] = toDist #다음 역까지의 최소소요시간을 더 짧은 것으로 업데이트
            routing[toGo]['route'] = copy.deepcopy(routing[visit]['route']) #다음역의 route도 지금까지 방문한 역의 route를 복사해줌
            routing[toGo]['route'].append(visit) #다음역의 route에 방문한 역을 추가함.
            

In [2]:
#최단경로 알고리즘
import copy #deepcopy를 하기 위해.
import sys

#지하철 정보 : 실제 네이버 노선도를 참고하여 걸리는 시간을 적었다.
stations = {
    #2호선 라인 - 충정로부터 반시계로 돌아가며 기입, 연결라인은 다음 역, 이전 역, 환승라인 순서.
    '충정로':             {'합정':10, '시청':2, '공덕':3, '을지로4가':8},
    '합정' :        {'당산':3 ,'충정로':10, '공덕':7},
    '당산' :     {'영등포구청':2, '합정':3, '여의도':4},
    '영등포구청':   {'신도림':4, '당산':2, '신길':4},
    '신도림' :      {'대림':3, '영등포구청':4, '신길':5},
    '대림' :            {'사당':14, '신도림':3, '이수':17},
    '사당' :          {'교대':8, '대림':14, '이수':2},
    '교대' :          {'선릉':14, '사당':8, '고속터미널역':2},
    '선릉' :          {'종합운동장':5, '교대':14, '강남구청':3},
    '종합운동장' :          {'건대입구':13, '선릉':14, '고속터미널역':15},
    '건대입구' :          {'왕십리':7, '종합운동장':13, '강남구청':7},
    '왕십리' :          {'신당':3, '건대입구':7, '강남구청':7, '동대문역사문화공원':7},
    '신당' :          {'동대문역사문화공원':2, '왕십리':3, '삼각지':13},
    '동대문역사문화공원' :          {'을지로4가':2, '신당':2, '충무로':3},
    '을지로4가' :          {'을지로3가':1, '동대문역사문화공원':2, '충정로':8},
    '을지로3가' :          {'시청':4, '을지로4가':1, '충무로':2},
    '시청' :          {'충정로':2, '을지로3가':4, '서울역':2},
    #2호선 이외 라인 - 서울역부터 아래로 내려가며 끝까지 가면 우측 맨위로 가며 기입, 연결라인은 12시부터 시계방향으로 
    '서울역' :          {'시청':2, '충무로':5, '삼각지':4, '노량진':9},
    '공덕' :          {'충정로':3, '삼각지':4, '여의도':6, '합정':7},
    '여의도' :          {'공덕':4, '노량진':3, '신길':2, '당산':4},
    '신길' :          {'여의도':2, '노량진':4, '신도림':5, '영등포구청':4},
    '노량진' :          {'서울역':9, '동작':5, '신길':4, '여의도':3},
    '삼각지' :          {'서울역':4, '신당':13, '동작':7, '공덕':4},
    '동작' :          {'삼각지':7, '고속터미널역':5, '이수':3, '노량진':5},
    '이수' :          {'동작':3, '고속터미널역':5, '사당':2, '대림':17},
    '충무로' :          {'을지로3가':2, '동대문역사문화공원':3, '고속터미널역':16, '서울역':5},
    '고속터미널역' :          {'충무로':16, '강남구청':7, '종합운동장':15, '교대':2, '이수':5, '동작':5},
    '강남구청' :          {'왕십리':7, '건대입구':7, '선릉':3, '고속터미널역':7}
    }

routing = {}
#모든 역들의 routing을 초기화.
for station in stations.keys():
    routing[station]={'shortestTime':0, 'route':[], 'visited':0}
   
print("============[지하철 노선도]============")    


# 역을 표시해주기. (bfs는 list를 return.)
print("이용가능한 역: " ,bfs(stations, '충정로'))


#목적지와 도착지를 설정해준다.
print("출발지를 입력하세요: ")
departure = input()

#없는 역을 입력시.
if departure in stations:
    pass
else:
    print("ERROR : 없는 역입니다.")
    sys.exit(1)

print("도착지를 입력하세요: ")
destination = input()

#없는 역을 입력시.
if destination in stations:
    pass
else:
    print("ERROR : 없는 역입니다.")
    sys.exit(1)

#2 출발지를 기준으로하는 초기 연결상태 리스트를 만든다.           
visitStation(departure)
print("========")

"""
아래부터의 구동과정

1 소요시간이 가장 짧은 역부터 긴 역 순으로 방문하고 방문한 역은 visited을 1로 초기화하여 표시한다.

2 새로운 역을 방문하면 그 역과 이어진 역들까지의 소요시간을 수정한다. 단, 이전에 이미 최소 소요시간이 구해졌다면 소요시간을 서로 비요하여 작은 것으로 바꾸거나 유지한다.

3 그래프의 모든 역들을 방문할 때 까지 1, 2과정을 반복한다.
"""

while 1 :
    #1
    #routing딕셔너리 안의 shortestTime중 가장 큰 값을 minTime에 넣는다. routing 딕셔너리안에 들어있는 요소들 중 값을 뽑아와서 (좌항), 그것들 하나하나의 shortestTime를 뽑아온다.(우항)-> 그 중 가장 max값이 있는 values 딕셔너리를 가져오는데 그 중 [shortestTime]를 가져온다.
    minTime = max(routing.values(), key=lambda x:x['shortestTime'])['shortestTime']
    toVisit = ''
    for name, search in routing.items(): #name은 키값(역 이름), search는 벨류값(정보들))
        if 0 < search['shortestTime'] <= minTime and not search['visited']: #minTime보다 해당 역까지의 최소소요시간이 작고 방문이 안되어있다면 해당역까지 방문하도록한다.
            minTime = search['shortestTime'] #이걸로 minTime를 계속 update하기에 loop한 번 돌 때마다 가장 작은 shortestTime가 minTime가 된다. 위에 minTime초기화 할 때가 아니라 여기서!!!!!
            toVisit = name #가장 작은 shortestTime를 가진 역을 다음 방문 역으로 둔다. 
            
    #3 # 방문할 게 안남으면 탈출한다.
    if toVisit == '':
        break
        
    #2 방문한다. (S에 포함시킨다.)
    visitStation(toVisit)

    print ("["+toVisit+"]")
    print ("Dist :", minTime)
    print("========")

#경로 리스트에 도착역을 추가하여 완전한 경로로 만들어주기.
routing[destination]['route'].append(destination)

print ("\n", "[", departure, "->", destination,"]")
print ("경로 :", routing[destination]['route'])
print ("소요시간 :", routing[destination]['shortestTime'], "분이 소요됩니다.")


============[지하철 노선도]============
이용가능한 역:  ['충정로', '합정', '시청', '공덕', '을지로4가', '당산', '을지로3가', '서울역', '삼각지', '여의도', '동대문역사문화공원', '영등포구청', '충무로', '노량진', '신당', '동작', '신길', '신도림', '고속터미널역', '왕십리', '이수', '대림', '강남구청', '종합운동장', '교대', '건대입구', '사당', '선릉']
출발지를 입력하세요: 
합정
도착지를 입력하세요: 
공덕
[당산]
Dist : 3
[영등포구청]
Dist : 5
[여의도]
Dist : 7
[공덕]
Dist : 7
[신길]
Dist : 9
[신도림]
Dist : 9
[노량진]
Dist : 10
[충정로]
Dist : 10
[삼각지]
Dist : 11
[시청]
Dist : 12
[대림]
Dist : 12
[서울역]
Dist : 14
[동작]
Dist : 15
[을지로3가]
Dist : 16
[을지로4가]
Dist : 17
[충무로]
Dist : 18
[이수]
Dist : 18
[동대문역사문화공원]
Dist : 19
[고속터미널역]
Dist : 20
[사당]
Dist : 20
[신당]
Dist : 21
[교대]
Dist : 22
[왕십리]
Dist : 24
[강남구청]
Dist : 27
[선릉]
Dist : 30
[건대입구]
Dist : 31
[종합운동장]
Dist : 35

 [ 합정 -> 공덕 ]
경로 : ['합정', '공덕']
소요시간 : 7 분이 소요됩니다.


# 지하철 역 정보
충정로 -
합정 -
당산 -
영등포구청 -
신도림 -
대림 -
사당 -
교대 -
선릉 -
종합운동장 -
건대입구 -
왕십리 -
신당 -
동대문역사문화공원 -
을지로4가 -
을지로3가 -
시청 -

서울역 -
공덕 -
여의도 -
신길 -
노량진 -
삼각지 -
동작 -
이수 -
충무로 -
고속터미널역 -
강남구청 -

stations = {
    #2호선 라인 - 반시계로 역하나
    '충정로':             {'합정':10, '시청':4, '공덕':3, '을지로4가':8},
    '합정' :        {'당산':3 ,'충정로':10, '공덕':7},
    '당산' :     {'영등포구청':2, '합정':3, '여의도':4},
    '영등포구청':   {'신도림':4, '당산':2, '신길':4},
    '신도림' :      {'대림':3, '영등포구청':4, '신길':5},
    '대림' :            {'사당':14, '신도림':3, '이수':17},
    '사당' :          {'교대':8, '대림':14, '이수':2},
    '교대' :          {'선릉':14, '사당':8, '고속터미널역':2},
    '선릉' :          {'종합운동장':5, '교대':14, '강남구청':3},
    '종합운동장' :          {'건대입구':13, '선릉':14, '고속터미널역':15},
    '건대입구' :          {'왕십리':7, '종합운동장':13, '강남구청':7},
    '왕십리' :          {'신당':3, '건대입구':7, '강남구청':7, '동대문역사문화공원':7},
    '신당' :          {'동대문역사문화공원':2, '왕십리':3, '삼각지':13},
    '동대문역사문화공원' :          {'을지로4가':2, '신당':2, '충무로':3},
    '을지로4가' :          {'을지로3가':1, '동대문역사문화공원':2, '충정로':8},
    '을지로3가' :          {'시청':4, '을지로4가':1, '충무로':2},
    '시청' :          {'충정로':2, '을지로3가':4, '서울역':2},
    #2호선 이외 라인 - 12시부터 시계방향으로 
    '서울역' :          {'시청':2, '충무로':5, '삼각지':4, '노량진':9},
    '공덕' :          {'충정로':3, '삼각지':4, '여의도':6, '합정':7},
    '여의도' :          {'공덕':4, '노량진':3, '신길':2, '당산':4},
    '신길' :          {'여의도':2, '노량진':4, '신도림':5, '영등포구청':4},
    '노량진' :          {'서울역':9, '동작':5, '신길':4, '여의도':3},
    '삼각지' :          {'서울역':4, '신당':13, '동작':7, '공덕':4},
    '동작' :          {'삼각지':7, '고속터미널역':5, '이수':3, '노량진':5},
    '이수' :          {'동작':3, '고속터미널역':5, '사당':2, '대림':17},
    '충무로' :          {'을지로3가':2, '동대문역사문화공원':3, '고속터미널역':16, '서울역':5},
    '고속터미널역' :          {'충무로':16, '강남구청':7, '종합운동장':15, '교대':2, '이수':5, '동작':5},
    '강남구청' :          {'왕십리':7, '건대입구':7, '선릉':3, '고속터미널역':7}
    }

